In [1]:
%cd ..

/home/hien/Documents/models/reid/rep_reid


# Triplet Loss

## Hard Mining triplet loss

In [ ]:
# TODO:

## Weighted Regularization Triplet

In [2]:
import torch

from pytorch_metric_learning.utils import common_functions as c_f

embedding_angles = [0, 20, 40, 60, 80]

embeddings = torch.tensor(
    [c_f.angle_to_coord(a) for a in embedding_angles],
    requires_grad=True,
    dtype=torch.float32,
)
labels = torch.LongTensor([0, 0, 1, 1, 2])
embeddings, labels

(tensor([[1.0000, 0.0000],
         [0.9397, 0.3420],
         [0.7660, 0.6428],
         [0.5000, 0.8660],
         [0.1736, 0.9848]], requires_grad=True),
 tensor([0, 0, 1, 1, 2]))

In [4]:
from src.metrics import self_cosine_dist, self_euclidean_dist, self_cosine_similarity

dist_mat = self_euclidean_dist(embeddings)
dist_mat

tensor([[0.0000, 0.1206, 0.4679, 1.0000, 1.6527],
        [0.1206, 0.0000, 0.1206, 0.4679, 1.0000],
        [0.4679, 0.1206, 0.0000, 0.1206, 0.4679],
        [1.0000, 0.4679, 0.1206, 0.0000, 0.1206],
        [1.6527, 1.0000, 0.4679, 0.1206, 0.0000]], grad_fn=<SubBackward0>)

In [5]:
self_cosine_similarity(embeddings)


tensor([[1.0000, 0.9397, 0.7660, 0.5000, 0.1736],
        [0.9397, 1.0000, 0.9397, 0.7660, 0.5000],
        [0.7660, 0.9397, 1.0000, 0.9397, 0.7660],
        [0.5000, 0.7660, 0.9397, 1.0000, 0.9397],
        [0.1736, 0.5000, 0.7660, 0.9397, 1.0000]], grad_fn=<MmBackward>)

In [6]:
self_cosine_dist(embeddings)

tensor([[ 0.0000e+00,  6.0307e-02,  2.3396e-01,  5.0000e-01,  8.2635e-01],
        [ 6.0307e-02,  0.0000e+00,  6.0307e-02,  2.3396e-01,  5.0000e-01],
        [ 2.3396e-01,  6.0307e-02,  0.0000e+00,  6.0307e-02,  2.3396e-01],
        [ 5.0000e-01,  2.3396e-01,  6.0307e-02,  0.0000e+00,  6.0307e-02],
        [ 8.2635e-01,  5.0000e-01,  2.3396e-01,  6.0307e-02, -1.1921e-07]],
       grad_fn=<RsubBackward1>)

In [7]:
import torch.nn.functional as F
embeddings2 = F.normalize(embeddings, p=2, dim=1)
embeddings2

tensor([[1.0000, 0.0000],
        [0.9397, 0.3420],
        [0.7660, 0.6428],
        [0.5000, 0.8660],
        [0.1736, 0.9848]], grad_fn=<DivBackward0>)

In [8]:
self_euclidean_dist(embeddings2)

tensor([[0.0000, 0.1206, 0.4679, 1.0000, 1.6527],
        [0.1206, 0.0000, 0.1206, 0.4679, 1.0000],
        [0.4679, 0.1206, 0.0000, 0.1206, 0.4679],
        [1.0000, 0.4679, 0.1206, 0.0000, 0.1206],
        [1.6527, 1.0000, 0.4679, 0.1206, 0.0000]], grad_fn=<SubBackward0>)

In [9]:
self_cosine_similarity(embeddings2)

tensor([[1.0000, 0.9397, 0.7660, 0.5000, 0.1736],
        [0.9397, 1.0000, 0.9397, 0.7660, 0.5000],
        [0.7660, 0.9397, 1.0000, 0.9397, 0.7660],
        [0.5000, 0.7660, 0.9397, 1.0000, 0.9397],
        [0.1736, 0.5000, 0.7660, 0.9397, 1.0000]], grad_fn=<MmBackward>)

In [10]:
self_cosine_dist(embeddings2)

tensor([[ 0.0000e+00,  6.0307e-02,  2.3396e-01,  5.0000e-01,  8.2635e-01],
        [ 6.0307e-02,  0.0000e+00,  6.0307e-02,  2.3396e-01,  5.0000e-01],
        [ 2.3396e-01,  6.0307e-02,  0.0000e+00,  6.0307e-02,  2.3396e-01],
        [ 5.0000e-01,  2.3396e-01,  6.0307e-02,  0.0000e+00,  6.0307e-02],
        [ 8.2635e-01,  5.0000e-01,  2.3396e-01,  6.0307e-02, -1.1921e-07]],
       grad_fn=<RsubBackward1>)

In [4]:
batch_size = 5
targets = labels.view(batch_size, 1).expand(batch_size, batch_size)

is_pos = targets.eq(targets.t()).float()
is_neg = targets.ne(targets.t()).float()
is_pos, is_neg

(tensor([[1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [0., 0., 1., 1., 0.],
         [0., 0., 1., 1., 0.],
         [0., 0., 0., 0., 1.]]),
 tensor([[0., 0., 1., 1., 1.],
         [0., 0., 1., 1., 1.],
         [1., 1., 0., 0., 1.],
         [1., 1., 0., 0., 1.],
         [1., 1., 1., 1., 0.]]))

In [22]:
dist_ap = dist_mat * is_pos
dist_an = dist_mat * is_neg
dist_ap, dist_an

(tensor([[0.0000, 0.1206, 0.0000, 0.0000, 0.0000],
         [0.1206, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.1206, 0.0000],
         [0.0000, 0.0000, 0.1206, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]], grad_fn=<MulBackward0>),
 tensor([[0.0000, 0.0000, 0.4679, 1.0000, 1.6527],
         [0.0000, 0.0000, 0.1206, 0.4679, 1.0000],
         [0.4679, 0.1206, 0.0000, 0.0000, 0.4679],
         [1.0000, 0.4679, 0.0000, 0.0000, 0.1206],
         [1.6527, 1.0000, 0.4679, 0.1206, 0.0000]], grad_fn=<MulBackward0>))

### softmax_weights

In [23]:
max_v = torch.max(dist_ap * is_pos, dim=1, keepdim=True)[0]
max_v

tensor([[0.1206],
        [0.1206],
        [0.1206],
        [0.1206],
        [0.0000]], grad_fn=<MaxBackward0>)

In [24]:
diff = dist_ap - max_v
diff

tensor([[-0.1206,  0.0000, -0.1206, -0.1206, -0.1206],
        [ 0.0000, -0.1206, -0.1206, -0.1206, -0.1206],
        [-0.1206, -0.1206, -0.1206,  0.0000, -0.1206],
        [-0.1206, -0.1206,  0.0000, -0.1206, -0.1206],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]], grad_fn=<SubBackward0>)

In [27]:
torch.sum(torch.exp(diff) * is_pos, dim=1, keepdim=True) + 1e-6


tensor([[1.8864],
        [1.8864],
        [1.8864],
        [1.8864],
        [1.0000]], grad_fn=<AddBackward0>)

In [28]:
torch.exp(diff) * is_pos / (
    torch.sum(torch.exp(diff) * is_pos, dim=1, keepdim=True) + 1e-6
)


tensor([[0.4699, 0.5301, 0.0000, 0.0000, 0.0000],
        [0.5301, 0.4699, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.4699, 0.5301, 0.0000],
        [0.0000, 0.0000, 0.5301, 0.4699, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.0000]], grad_fn=<DivBackward0>)

In [29]:
def softmax_weights(dist, mask, eps=1e-6):
    max_v = torch.max(dist * mask, dim=1, keepdim=True)[0]

    diff = dist - max_v

    Z = torch.sum(torch.exp(diff) * mask, dim=1, keepdim=True) + eps

    W = torch.exp(diff) * mask / Z

    return W


### Weight example mining

In [30]:
weights_ap = softmax_weights(dist_ap, is_pos)
weights_an = softmax_weights(-dist_an, is_neg)
weights_ap, weights_an

(tensor([[0.4699, 0.5301, 0.0000, 0.0000, 0.0000],
         [0.5301, 0.4699, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.4699, 0.5301, 0.0000],
         [0.0000, 0.0000, 0.5301, 0.4699, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 1.0000]], grad_fn=<DivBackward0>),
 tensor([[0.0000, 0.0000, 0.5282, 0.3103, 0.1615],
         [0.0000, 0.0000, 0.4713, 0.3330, 0.1956],
         [0.2928, 0.4144, 0.0000, 0.0000, 0.2928],
         [0.1956, 0.3330, 0.0000, 0.0000, 0.4713],
         [0.0924, 0.1775, 0.3023, 0.4278, 0.0000]], grad_fn=<DivBackward0>))

In [31]:
dist_ap = torch.sum(dist_ap * weights_ap, dim=1)
dist_an = torch.sum(dist_an * weights_an, dim=1)
dist_ap, dist_an

(tensor([0.0639, 0.0639, 0.0639, 0.0639, 0.0000], grad_fn=<SumBackward1>),
 tensor([0.8244, 0.4083, 0.3240, 0.4083, 0.5233], grad_fn=<SumBackward1>))

# Multi similarity loss

In [3]:
import torch

from pytorch_metric_learning.utils import common_functions as c_f

embedding_angles = [0, 20, 40, 60, 80]

embeddings = torch.tensor(
    [c_f.angle_to_coord(a) for a in embedding_angles],
    requires_grad=True,
    dtype=torch.float32,
)
labels = torch.LongTensor([0, 0, 1, 1, 2])
embeddings, labels

(tensor([[1.0000, 0.0000],
         [0.9397, 0.3420],
         [0.7660, 0.6428],
         [0.5000, 0.8660],
         [0.1736, 0.9848]], requires_grad=True),
 tensor([0, 0, 1, 1, 2]))

In [4]:
from pytorch_metric_learning.losses import MultiSimilarityLoss

alpha, beta, base = 0.1, 10, 0.5

loss_func = MultiSimilarityLoss(alpha=alpha, beta=beta, base=base)

loss_func(embeddings, labels)

tensor(5.7961, grad_fn=<MeanBackward0>)

In [6]:
from src.losses.multi_similarity_loss import MultiSimilarityLoss

loss_func = MultiSimilarityLoss(alpha=alpha, beta=beta, gamma=base, eps=1e-12)

loss_func(embeddings, labels)

tensor(5.7961, grad_fn=<MeanBackward0>)

# Circle loss

In [7]:
import torch

from pytorch_metric_learning.utils import common_functions as c_f

embedding_angles = [0, 20, 40, 60, 80]

embeddings = torch.tensor(
    [c_f.angle_to_coord(a) for a in embedding_angles],
    requires_grad=True,
    dtype=torch.float32,
)
labels = torch.LongTensor([0, 0, 1, 1, 2])
embeddings, labels

(tensor([[1.0000, 0.0000],
         [0.9397, 0.3420],
         [0.7660, 0.6428],
         [0.5000, 0.8660],
         [0.1736, 0.9848]], requires_grad=True),
 tensor([0, 0, 1, 1, 2]))

In [8]:
from pytorch_metric_learning.losses import CircleLoss

margin, gamma = 0.25, 256

loss_func = CircleLoss(m=margin, gamma=gamma)

loss_func(embeddings, labels)

tensor(176.0281, grad_fn=<MeanBackward0>)

In [10]:
from src.losses.circle_loss import CircleLoss

loss_func = CircleLoss(margin=margin, gamma=gamma)

loss_func(embeddings, labels)

tensor(176.0281, grad_fn=<DivBackward0>)

# Cosface loss

In [11]:
import torch

from pytorch_metric_learning.utils import common_functions as c_f

embedding_angles = [0, 20, 40, 60, 80]

embeddings = torch.tensor(
    [c_f.angle_to_coord(a) for a in embedding_angles],
    requires_grad=True,
    dtype=torch.float32,
)
labels = torch.LongTensor([0, 0, 1, 1, 2])
embeddings, labels



(tensor([[1.0000, 0.0000],
         [0.9397, 0.3420],
         [0.7660, 0.6428],
         [0.5000, 0.8660],
         [0.1736, 0.9848]], requires_grad=True),
 tensor([0, 0, 1, 1, 2]))

In [12]:
from src.losses.cosface_loss import CosFace

margin, gamma = 0.25, 256

loss_func = CosFace(margin=margin, gamma=gamma)

loss_func(embeddings, labels)

tensor(52.8865, grad_fn=<DivBackward0>)